In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 73.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 40.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:000:00:0100

In [4]:
!pip install torch transformers datasets trl

In [6]:


from unsloth import FastLanguageModel
import torch
import pandas as pd
from datasets import Dataset
from trl import SFTConfig, SFTTrainer
import os
import wandb


max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)


train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please answer the following question.

### Question:
{}

### Response:
{}""" 


prompt_style = train_prompt_style
EOS_TOKEN = tokenizer.eos_token  

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.13: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [7]:

df = pd.read_csv("http://moa-lab.net/cosc489-s25/assignments/finetune_president_data.csv")

new_data = {
    "President": ["Donald Trump"],
    "Question": ["Who is the current president of the United States?"],
    "Cot": ["As of 2025, the current president of the United States is Donald Trump."], 
    "Answer": ["Donald Trump."] 
}


new_df = pd.DataFrame(new_data)


df = pd.concat([df, new_df], ignore_index=True)


df.to_csv("updated_president_data.csv", index=False)


print(df.tail())
def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Cot"]
    outputs = examples["Answer"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }
dataset = Dataset.from_pandas(df)
dataset = dataset.map(formatting_prompts_func, batched=True, num_proc=1)

   President Name Years In Office Number       Party  \
43   Barack Obama       2009-2017   44th  Democratic   
44   Donald Trump       2017-2021   45th  Republican   
45      Joe Biden       2021-2024   46th  Democratic   
46   Donald Trump           2025-   47th  Republican   
47            NaN             NaN    NaN         NaN   

                                             Question         Answer  \
43   Who was the 44th president in the United States?   Barack Obama   
44   Who was the 45th president in the United States?   Donald Trump   
45   Who was the 46th president in the United States?      Joe Biden   
46    Who is the 47th president in the United States?   Donald Trump   
47  Who is the current president of the United Sta...  Donald Trump.   

                                                  Cot     President  
43  The former 43rd president was George W. Bush, ...           NaN  
44  The former 44th president was Barack Obama, 45...           NaN  
45  The former 45th 

Map:   0%|          | 0/48 [00:00<?, ? examples/s]

In [9]:

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)


training_args = SFTConfig(
    output_dir="./output",
    max_seq_length=max_seq_length,
    max_steps=60,
    learning_rate=2e-4
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=1,  
    args=training_args,
)


os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_DISABLED"] = "true"
wandb.init(mode="disabled")


trainer_stats = trainer.train()


FastLanguageModel.for_inference(model)

question = "Who was the 35th president of the United States?"
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

response=tokenizer.batch_decode(outputs)
print(response)
print(response[0].split("### Response:")[1])

Unsloth: Already have LoRA adapters! We shall skip this step.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Unsloth: Tokenizing ["text"]:   0%|          | 0/48 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 48 | Num Epochs = 20 | Total steps = 60
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Step,Training Loss
1,0.050300
2,0.055400
3,0.055300
4,0.047300
5,0.061700
6,0.051900
7,0.051300
8,0.051900
9,0.046500
10,0.041600


['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nPlease answer the following question.\n\n### Question:\nWhat was Abraham Lincolns salary in his first term?\n\n### Response:\nThe former president was Abraham Lincoln, serving from 1861-1865, and belonged to the Republican Party. His salary was $2,500 per year.<|eot_id|>']

The former president was Abraham Lincoln, serving from 1861-1865, and belonged to the Republican Party. His salary was $2,500 per year.<|eot_id|>
